In [17]:
import numpy as np
import pandas as pd 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import os

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/yaagocruuz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
imdb_dir = '/home/yaagocruuz/Documentos/git/Topicos3/aclImdb/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')
labels = []
texts = []
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [3]:
test_dir = os.path.join(imdb_dir, 'test')
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

# Remova
pontuações, stopwords e realize o processo de redução ao radical
(stremming)

In [4]:
stop = stopwords.words('english')
def ponctuation(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))    
    return text

In [5]:
for i in range(0,len(texts)):
    texts[i] = ponctuation(texts[i])

In [6]:
texts[0]

'the bloodsucker leads the dance what a laughable title it s so utterly misleading it s not surprising that the film makers try and mislead us though because this is one terrible movie the story basically involves a murder mystery in a castle on a remote island very little happens in this film and when something does wake the viewer from his stupor it invariably is unintentional comedy in the form of atrocious dialogue delivered by a hopeless group of voice artists these guys are so bad they make the actors they deliver voices for appear like a group of remedial level morons it really is hard to determine how bad the acting is when you have dubbing this abysmal but the voice artists cannot be blamed for the script it s a travesty unintentionally funny at best pathetic at worst the story in general is to say the least uneven the women characters are particularly idiotic the men are either creepy or tedious the whole enterprise smacks of pure exploitation of the audience it doesn t remot

In [7]:
stemmer = PorterStemmer()
def stemmering(text):
    return [stemmer.stem(w) for w in text.split()]

In [8]:
def stem(text):
    text_stem = [w for w in stemmering(text) if w not in stop]
    new_text = ''
    for i in text_stem[0:]:
        new_text = new_text + i + ' '
    return new_text

In [9]:
for i in range(0,len(texts)):
    texts[i] = stem(texts[i])

In [10]:
texts[0]

'bloodsuck lead danc laughabl titl utterli mislead surpris film maker tri mislead us though becaus thi one terribl movi stori basic involv murder mysteri castl remot island veri littl happen thi film someth doe wake viewer hi stupor invari unintent comedi form atroci dialogu deliv hopeless group voic artist guy bad make actor deliv voic appear like group remedi level moron realli hard determin bad act dub thi abysm voic artist cannot blame script travesti unintent funni best pathet worst stori gener say least uneven women charact particularli idiot men either creepi tediou whole enterpris smack pure exploit audienc remot deliv promis even murder final start happen occur screen get half heart sever head shot peopl said thi movi giallo cannot agre less thi opinion anyon enjoy italian thriller give thi movi wide berth noth remot thrill basic soft core porn film horror angl veri erot either recommend thi anyon '

# crie um classificador MultinomialNB
por meio da biblioteca scikit-learn

In [14]:
X_train = texts[:40000]
y_train = labels[:40000]
X_test = texts[:40000]
y_test = labels[:40000]

In [16]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_test_counts = count_vect.fit_transform(X_test)

In [20]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(40000, 65837)

In [21]:
X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)
X_test_tfidf.shape

(40000, 65837)

In [22]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [23]:
y_pred_mnb = clf.predict(X_test_tfidf)

In [25]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_mnb)

0.793275

# Questao 3

In [26]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore', n_features=2**21)

In [28]:
X = vect.fit_transform(X_train)

In [30]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier()

In [31]:
clf.fit(X, y_train)

/home/yaagocruuz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [32]:
y_pred_sgd = clf.predict(X)

In [33]:
metrics.accuracy_score(y_test,y_pred_sgd)

0.902225

# Questao 4

In [48]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB
clf_gaussian = GaussianNB()
clf_multinomial = MultinomialNB()
clf_bernoulli = BernoulliNB()
#NAO SEI PORQUE O GAUSSIAN E O MULTINOMIAL NAO FUNCIONAM

In [44]:
clf_bernoulli.fit(X_train_tfidf, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [46]:
y_pred_ber = clf_bernoulli.predict(X_test_tfidf)

In [47]:
metrics.accuracy_score(y_test, y_pred_ber)

0.874975